In [1]:
import numpy as np
from scipy.linalg import solve

# Define beam properties
length = 2.0  # Length of the beam (m)
num_elements = 128  # Number of elements
element_length = length / num_elements
E = 200e9  # Young's Modulus (Pa)
I = 4e-6  # Area moment of inertia (m^4)

# Define support locations
support_a = 0.0  # Position of fixed support A (m)
support_b = 1.0  # Position of roller support B (m)
support_c = 2.0  # Position of roller support C (m)

# Define distributed loads (UDL) in kN/m
udl_magnitude = 12  # kN/m

# Calculate the number of degrees of freedom (DOFs)
num_dofs = (num_elements + 1) * 2

# Create the global stiffness matrix and load vector
K_global = np.zeros((num_dofs, num_dofs))
F_global = np.zeros(num_dofs)

# Define the element stiffness matrix
element_stiffness = (E * I / element_length**3) * np.array([[12, 6 * element_length, -12, 6 * element_length],
                                                           [6 * element_length, 4 * element_length**2, -6 * element_length, 2 * element_length**2],
                                                           [-12, -6 * element_length, 12, -6 * element_length],
                                                           [6 * element_length, 2 * element_length**2, -6 * element_length, 4 * element_length**2]])

# Assemble the global stiffness matrix
for i in range(num_elements):
    start_dof = i * 2
    end_dof = start_dof + 4
    K_global[start_dof:end_dof, start_dof:end_dof] += element_stiffness

# Apply distributed loads (UDL) to the global load vector (only between B and C)
udl_force = udl_magnitude * element_length
start_element = int(support_b / element_length)
end_element = int(support_c / element_length)
for i in range(start_element, end_element):
    F_global[i * 2] += udl_force / 2  # Applied to the left node of the element
    F_global[i * 2 + 2] += udl_force / 2  # Applied to the right node of the element

# Boundary Conditions (fixed support at A, roller support at B, and constrain vertical displacement at C)
fixed_dof = [1]  # Constrain vertical displacement at supports B and C
fixed_dof.append(0)  # Constrain vertical displacement at A

# Remove rows and columns corresponding to boundary conditions
boundary_dof = fixed_dof
K_reduced = np.delete(np.delete(K_global, boundary_dof, axis=0), boundary_dof, axis=1)
F_reduced = np.delete(F_global, boundary_dof)

# Solve for displacements
displacements_reduced = solve(K_reduced, F_reduced)

# Calculate reactions
reactions = K_global.dot(np.insert(displacements_reduced, boundary_dof, [0, 0])) - F_global

# Calculate slopes at points B and C
slopes_at_b = displacements_reduced[int((support_b - element_length) / element_length) * 2 + 1]
slopes_at_c = displacements_reduced[int((support_c - element_length) / element_length) * 2 + 1]

# Calculate displacement at 1.5 m from the fixed support (Point D)
point_d = 1.5
dof_at_point_d = int((point_d - element_length) / element_length) * 2
displacement_at_point_d = displacements_reduced[dof_at_point_d]

# Output results
print("Slope at Point B:", slopes_at_b, "rad")
print("Slope at Point C:", slopes_at_c, "rad")
print("Displacement at 1.5 m from the fixed support (Point D):", displacement_at_point_d, "m")


Slope at Point B: 1.4999999978272236e-05 rad
Slope at Point C: 1.7500305147295326e-05 rad
Displacement at 1.5 m from the fixed support (Point D): 1.6914100622484965e-05 m


In [2]:
import numpy as np

# Material properties
E = 2.1e11  # Young's modulus in Pa
A = 0.01   # Cross-sectional area in m^2
L = 2.0    # Length of the beam in meters

# Element stiffness matrix for a 2D beam element
k = (E * A) / L
K_local = np.array([
    [k, 0, 0, -k, 0, 0],
    [0, 12 * E * I / L ** 3, 6 * E * I / L ** 2, 0, -12 * E * I / L ** 3, 6 * E * I / L ** 2],
    [0, 6 * E * I / L ** 2, 4 * E * I / L, 0, -6 * E * I / L ** 2, 2 * E * I / L],
    [-k, 0, 0, k, 0, 0],
    [0, -12 * E * I / L ** 3, -6 * E * I / L ** 2, 0, 12 * E * I / L ** 3, -6 * E * I / L ** 2],
    [0, 6 * E * I / L ** 2, 2 * E * I / L, 0, -6 * E * I / L ** 2, 4 * E * I / L]
])

# Nodal coordinates
nodes = [0, L]

# Number of nodes and degrees of freedom
num_nodes = len(nodes)
num_dofs = 6  # Six DOFs per element

# Initialize the global stiffness matrix
K_global = np.zeros((num_dofs, num_dofs))

# Assemble the global stiffness matrix
for i in range(num_nodes - 1):
    start = i * 3
    end = start + 6
    K_global[start:end, start:end] += K_local

# Print the global stiffness matrix
print("Global Stiffness Matrix:")
print(K_global)


Global Stiffness Matrix:
[[ 1.05e+09  0.00e+00  0.00e+00 -1.05e+09  0.00e+00  0.00e+00]
 [ 0.00e+00  1.26e+06  1.26e+06  0.00e+00 -1.26e+06  1.26e+06]
 [ 0.00e+00  1.26e+06  1.68e+06  0.00e+00 -1.26e+06  8.40e+05]
 [-1.05e+09  0.00e+00  0.00e+00  1.05e+09  0.00e+00  0.00e+00]
 [ 0.00e+00 -1.26e+06 -1.26e+06  0.00e+00  1.26e+06 -1.26e+06]
 [ 0.00e+00  1.26e+06  8.40e+05  0.00e+00 -1.26e+06  1.68e+06]]
